In [ ]:
! pip install -q gpt-2-simple

     |████████████████████████████████| 462 kB 18.5 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import csv
import tweepy # https://github.com/tweepy/tweepy
import numpy as np
import pandas as pd
import configparser
import gpt_2_simple as gpt2

In [ ]:
Handler = 'elonmusk'
path = 'drive/MyDrive/Colab Notebooks/COE494_Project'

In [ ]:
def authenticate(path = 'drive/MyDrive/Colab Notebooks/COE494_Project/'):
  # read config
  config = configparser.ConfigParser()
  config.read(path + 'config.ini')

  api_key = str(config['twitter']['api_key'])
  api_key_secret = str(config['twitter']['api_key_secret'])

  access_token = str(config['twitter']['access_token'])
  access_token_secret = str(config['twitter']['access_token_secret'])

  # authenticate
  auth = tweepy.OAuthHandler(api_key, api_key_secret)
  auth.set_access_token(access_token, access_token_secret)

  return tweepy.API(auth, wait_on_rate_limit = True)

In [ ]:
api = authenticate()

In [ ]:
# Tweet text pre-processing
def clean_tweet(tweet):
    stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]
    if type(tweet) == float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    return temp

In [ ]:
def get_all_tweets(handler):
    # Twitter only allows access to a users most recent 3240 tweets with this method
    print(f'Grabbing @{handler}\'s Tweets')
    #initialize a list to hold all the tweepy Tweets
    all_tweets = []  
    
    # make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = handler, count = 200, include_rts = False, tweet_mode = 'extended')
    
    # save most recent tweets
    all_tweets.extend(new_tweets)
    
    # save the id of the oldest tweet less one
    oldest = all_tweets[-1].id - 1
    
    # keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:        
        # all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = handler, count = 200, max_id = oldest, include_rts = False, tweet_mode = 'extended')
        # save most recent tweets
        all_tweets.extend(new_tweets)        
        # update the id of the oldest tweet less one
        oldest = all_tweets[-1].id - 1
        
    print(f"{len(all_tweets)} tweets downloaded...")    
    # transform the tweepy tweets into a 2D array that will populate the csv 
    out_tweets = [[tweet.id_str, tweet.created_at, tweet.full_text] for tweet in all_tweets]
    df = pd.DataFrame (out_tweets, columns = ["id", "time", "tweet"])
    df.to_csv(path + '/data/' + handler+'.csv')
    return df

In [ ]:
# tweets = get_all_tweets(Handler)
tweets = pd.read_csv(path + "/data/" + Handler + ".csv")
cleaned_tweets = pd.DataFrame([clean_tweet(tweet) for tweet in tweets.tweet], columns = ['tweet'])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,4,9,14,16,17,19,22,24,25,26,31,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Removing null and empty rows 
cleaned_tweets.tweet.replace('', np.nan, inplace=True)
cleaned_tweets.dropna(inplace = True)
cleaned_tweets

,tweet
0,please ignore prior tweets as that was someone...
1,so true
2,if you ever wanted know real truth about moon ...
3,walked around neighborhood recently rebuilt wi...
4,it was xmas so we brought presents kids at orp...
...,...
34870,reminds me when i hex edited ultima v get out ...
34871,yay switzerland
34872,there is no way be touch with voters when you ...
34874,let s make roaring 20 s happen


In [ ]:
tweets = cleaned_tweets['tweet'].tolist()

In [ ]:
# randomizing tweets to prevent overfitting
import random

ENDTOKEN = "<|endoftext|>"
EPOCHS = 10

with open("tweets.txt", "w") as fp:
    fp.write(ENDTOKEN)
    for _ in range(EPOCHS):
        random.shuffle(tweets)
        fp.write(ENDTOKEN.join(tweets) + ENDTOKEN)

In [ ]:
! pip install transformers

     |████████████████████████████████| 4.0 MB 12.4 MB/s 
     |████████████████████████████████| 880 kB 57.2 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 61.7 MB/s 
     |████████████████████████████████| 6.6 MB 57.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=56f2819cd5a253cfd3fa6ca48e035c35ac235ae70d3545d091419e42cc6ffc76
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TextDataset, DataCollatorForLanguageModeling,
    Trainer, TrainingArguments
)

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")
train_dataset = TextDataset(tokenizer = tokenizer, file_path="tweets.txt", block_size=tokenizer.model_max_length)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (4673311 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
training_args = TrainingArguments(
    output_dir='./output',
    overwrite_output_dir = True,
    num_train_epochs = 4,
    per_device_train_batch_size = 1,
    prediction_loss_only = True,
    logging_steps = 100,
    save_steps=0,
    seed = random.randint(0, 2**32-1),
)

trainer = Trainer(
    model = model,
    tokenizer = tokenizer,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset,
)

trainer.train()

Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (4673311 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4563
  Num Epochs = 4
  Instantaneous batch size per device = 1
  

Step,Training Loss
100,5.836900
200,5.409200
300,5.246200
400,5.106800
500,4.985500
600,4.874700
700,4.793000
800,4.700300
900,4.605000
1000,4.524000




Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in ./model/config.json
Model weights saved in ./model/pytorch_model.bin


In [ ]:
model.save_pretrained(path + "/models/2_GPT2")

Configuration saved in drive/MyDrive/Colab Notebooks/COE494_Project/models/2_GPT2/config.json
Model weights saved in drive/MyDrive/Colab Notebooks/COE494_Project/models/2_GPT2/pytorch_model.bin


In [ ]:
model = AutoModelForCausalLM.from_pretrained(path + "/models/2_GPT2")

In [ ]:
import string
# start_str = ENDTOKEN + random.choice(string.ascii_uppercase+string.digits)
start_str = ENDTOKEN + 'Roadster'
encoded_prompt = tokenizer(start_str, add_special_tokens=False, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(model.device)

output_sequences = model.generate(
    input_ids = encoded_prompt,
    max_length = 50,
    min_length = 10,
    temperature = 0.75,
    top_k = 100,
    top_p = 0.90,
    do_sample = True,
    num_return_sequences = 1,
    pad_token_id = tokenizer.eos_token_id # gets rid of warning
    )

for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    print(text.strip())

Roadster will have red calipers amp sport level brakes


In [ ]:
import string
# start_str = ENDTOKEN + random.choice(string.ascii_uppercase+string.digits)
start_str = ENDTOKEN + 'Finally'
encoded_prompt = tokenizer(start_str, add_special_tokens=False, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(model.device)

output_sequences = model.generate(
    input_ids = encoded_prompt,
    max_length = 50,
    min_length = 10,
    temperature = 0.75,
    top_k = 100,
    top_p = 0.90,
    do_sample = True,
    num_return_sequences = 1,
    pad_token_id = tokenizer.eos_token_id # gets rid of warning
    )

for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    print(text.strip())

Finally we will unveil fully functional advanced prototype model x almost identical production
